In [1]:
# !pip uninstall -y pinecone-client
!pip install --user pinecone-client[grpc]==3.0.0.dev4

In [2]:
!pip install pandas numpy sentence_transformers pyarrow

In [3]:
!pip install python-dotenv

In [4]:
!pip install --user tenacity

In [5]:
import ray
import os
import pandas as pd
import uuid
import json
import numpy as np
from time import time
from random import shuffle
from datetime import datetime
from tenacity import retry, stop_after_attempt, wait_exponential
import logging

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
ray.init(
    runtime_env={
        "env_vars": {
          "PINECONE_API_KEY": os.getenv("PINECONE_API_KEY"),
        }
    },
)

2024-01-12 12:29:33,739	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 10.0.0.33:6379...
2024-01-12 12:29:33,746	INFO worker.py:1715 -- Connected to Ray cluster. View the dashboard at https://session-tq9iqwenrl97au1eh61tat4fii.i.anyscaleuserdata.com 
2024-01-12 12:29:33,753	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_8d24b4baf867aea16b05f81a9b5ebdc7.zip' (0.73MiB) to Ray cluster...
2024-01-12 12:29:33,755	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_8d24b4baf867aea16b05f81a9b5ebdc7.zip'.


Python version:,3.10.13
Ray version:,2.9.0
Dashboard:,http://session-tq9iqwenrl97au1eh61tat4fii.i.anyscaleuserdata.com


# Load data

In [8]:
import requests
import json

url = "https://huggingface.co/api/datasets/Cohere/wikipedia-22-12-en-embeddings/parquet/default/train"
response = requests.get(url)
input_files = json.loads(response.content)
columns = ['id', 'title', 'text', 'url', 'emb',] 
ds = ray.data.read_parquet(input_files, columns=columns)

Metadata Fetch Progress 0:   0%|          | 0/42 [00:00<?, ?it/s]

Parquet Files Sample 0:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
print(f"Dataset size: {ds.count():,} vectors")

Dataset size: 35,167,920 vectors


In [10]:
# Replace these with your actual column name:
text_column = 'text'       
title_column = 'title' 
url_column = 'url'
embeddings_columm = 'emb'

def process_data(df):
    # Id column needs to be string
    df['id'] = df['id'].astype(str)
    
    # Rename the vector values column
    df.rename(columns={embeddings_columm: 'values'}, inplace=True)
    
    # Combine remaining columns to metadata
    metadata_cols = [text_column, title_column, url_column]
    df['metadata'] = df[metadata_cols].to_dict(orient='records')
    df.drop(columns=metadata_cols, inplace=True)

    return df



For debugging and visualization only - apply the processing to 100 lines and read to df.

In [11]:
# For debugging and visualization only:

df = ds.map_batches(process_data, batch_format="pandas").take_batch(100, batch_format="pandas")
df.head()

2024-01-12 12:30:40,714	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=1879 for stage ReadParquet to satisfy output blocks of size at most DataContext.get_current().target_max_block_size=128.0MiB.
2024-01-12 12:30:40,714	INFO set_read_parallelism.py:122 -- To satisfy the requested parallelism of 1879, each read task output is split into 2 smaller blocks.
2024-01-12 12:30:40,715	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> TaskPoolMapOperator[MapBatches(process_data)] -> LimitOperator[limit=100]
2024-01-12 12:30:40,716	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-01-12 12:30:40,716	INFO streaming_executor.py:115 -- Tip:

Running 0:   0%|          | 0/253 [00:00<?, ?it/s]

(ReadParquet->SplitBlocks(2) pid=13562, ip=10.0.0.63) /home/ray/anaconda3/lib/python3.10/site-packages/ray/data/_internal/arrow_block.py:148: FutureWarning: promote has been superseded by mode='default'.
(ReadParquet->SplitBlocks(2) pid=13562, ip=10.0.0.63)   return transform_pyarrow.concat(tables)


(autoscaler +1m13s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +1m13s) [autoscaler] [worker-node-type-0] Upscaling 1 node(s) to satisfy cluster resource constraints.


,id,values,metadata
0,2363068,"[0.17667872, -0.13867834, -0.021413453, 0.2931...",{'text': 'Social conflicts among cats depend s...
1,2363069,"[0.044649594, 0.15593667, 0.35089195, 0.345184...",{'text': 'The first sign of an imminent tomcat...
2,2363070,"[0.11803974, 0.16034317, -0.11615362, 0.473689...",{'text': 'Fights can also happen between two f...
3,2363071,"[0.2019098, 0.24923877, 0.053239096, 0.2989406...",{'text': 'Cats between the age of three and ni...
4,2363072,"[0.1188415, 0.36787227, 0.1163324, 0.13749832,...",{'text': 'Feral kittens around two to seven we...


In [12]:
df.metadata.iloc[0]

{'text': 'Social conflicts among cats depend solely on the behavior of the cats. Some research has shown that cats rarely pick fights, but when they do, it is usually for protecting food and/or litters, and defending their territory.',
 'title': 'Cat communication',
 'url': 'https://en.wikipedia.org/wiki?curid=5741167'}

# Upsert

In [13]:
from pinecone.grpc import PineconeGRPC
from pinecone import Pinecone, ServerlessSpec
import os
from datetime import datetime

In [14]:
pc = PineconeGRPC()

In [15]:
index_name = 'cohere-test-2'

In [16]:
# pc.delete_index(index_name)

In [17]:
indexes = pc.list_indexes().indexes
names = [_['name'] for _ in indexes]
names

['serverless-index',
 'cohere-wiki-full-2',
 'serverless-index-parallel',
 'vercel-chatbot-serverless',
 'anyscale-cohere-wiki',
 'test',
 'cohere-test',
 'cohere-wiki-full',
 'serverless-test']

In [18]:
if index_name not in names:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-west-2') 
    ) 

In [19]:
indexes = pc.list_indexes().indexes
[_ for _ in indexes if _['name'] == index_name]

[{'dimension': 768,
  'host': 'cohere-test-2-pltuh67.svc.apw5-4e34-81fa.pinecone.io',
  'metric': 'cosine',
  'name': 'cohere-test-2',
  'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
  'status': {'ready': True, 'state': 'Ready'}}]

In [20]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

## Sync option

Note: this option is more readable but much slower (unless you use x10 times more cpus...)

In [ ]:
batch_size = 350

def upload(batch):
    client = PineconeGRPC()
    index = client.Index(index_name)
    total_vectors = 0
    num_failures = 0
    # data = process_data(large_batch).to_dict(orient='records')
    data = batch.to_dict(orient='records')
    
    @retry(stop=stop_after_attempt(2), wait=wait_exponential(multiplier=1, min=4, max=10))
    def send_batch(batch):
        return index.upsert(vectors=batch)
    
    try:
        result = send_batch(data)
        total_vectors += result.upserted_count
    except Exception as e:
        logging.exception(e)
        num_failures += len(data)
    return {'upsreted': np.array([total_vectors]), 'errors': np.array([num_failures])}

class Upserter:
    def __call__(self, large_batch):
        return upload(large_batch)

In [ ]:
%%time

num_workers = 150

new_ds = ds.map_batches(
    process_data, 
    batch_format="pandas"
).map_batches(
    Upserter, 
    batch_size=batch_size, 
    batch_format='pandas', 
    zero_copy_batch=True,
    concurrency=(1, num_workers)
)


before = datetime.now()
summary = new_ds.sum(['upsreted', 'errors'])
duration = datetime.now() - before
print({k: f"{v: ,}" for k,v in summary.items()})
print(duration)

In [ ]:
print(f"Upserted {summary['sum(upsreted)']:,} vectors, duration: {duration}. Failed to upsert {summary['sum(errors)']:,} vectors, see log for more details")

## Async option

This version is not as straight forward and readable as the sync option, but much much faster.

However - it's also much more likely to hit the admission control on the Pinecone server side. If you're seeing too many errors (`"Too many requests. Please retry shortly"`) - try reducing `num_workers` and\or `concurrent_reqests`.

In [21]:
from threading import Semaphore

In [22]:
batch_size = 350
concurrent_reqests = 5

def chunker(seq, batch_size):
    return ((pos, seq[pos:pos + batch_size]) for pos in range(0, len(seq), batch_size))


def async_upload(large_batch):
    client = PineconeGRPC()
    index = client.Index(index_name)
    total_vectors = 0
    num_failures = 0
    data = large_batch.to_dict(orient='records')

    # The current implementation of `upsert(async_req=True)` uses thread pool, not asyncio - so we need threading Semaphore
    sem = Semaphore(concurrent_reqests)

    def done(future):
        sem.release()
    
    def send_batch(batch):
        sem.acquire()
        future = index.upsert(vectors=batch, async_req=True)
        future.add_done_callback(done)
        return future
    
    tasks = [(pos, send_batch(batch)) for pos, batch in chunker(data, batch_size=batch_size)] 
    total_vectors = 0
    failed_ids = []
    num_failures = 0
    for idx, task in tasks:
        try:
            result = task.result()
            total_vectors += result.upserted_count
        except Exception:
            failed_batch = data[idx: idx + batch_size]
            try:
                result = index.upsert(failed_batch)
                total_vectors += result.upserted_count
            except Exception as e:
                logging.exception(e)
                num_failures += len(failed_batch)
    return {'upsreted': np.array([total_vectors]), 'errors': np.array([num_failures])}
    
class UpserterConcurrent:
    def __call__(self, large_batch):
        return async_upload(large_batch)

In [23]:
%%time

num_workers = 5

new_ds = ds.map_batches(
    process_data, 
    batch_format="pandas"
).map_batches(
    UpserterConcurrent, 
    batch_size=batch_size * 20, 
    batch_format='pandas', 
    zero_copy_batch=True,
    concurrency=(1, num_workers)
)

before = datetime.now()
summary = new_ds.materialize().sum(['upsreted', 'errors'])
# summary
duration = datetime.now() - before
print({k: f"{v: ,}" for k,v in summary.items()})
print(duration)

2024-01-12 12:30:53,848	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=1879 for stage ReadParquet to satisfy output blocks of size at most DataContext.get_current().target_max_block_size=128.0MiB.
2024-01-12 12:30:53,849	INFO set_read_parallelism.py:122 -- To satisfy the requested parallelism of 1879, each read task output is split into 2 smaller blocks.
2024-01-12 12:30:53,849	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> ActorPoolMapOperator[MapBatches(process_data)->MapBatches(UpserterConcurrent)]
2024-01-12 12:30:53,850	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-01-12 12:30:53,851	INFO streaming_executor.py:115 --

Running 0:   0%|          | 0/253 [00:00<?, ?it/s]

(ReadParquet->SplitBlocks(2) pid=44152, ip=10.0.0.62) /home/ray/anaconda3/lib/python3.10/site-packages/ray/data/_internal/arrow_block.py:148: FutureWarning: promote has been superseded by mode='default'.
(ReadParquet->SplitBlocks(2) pid=44152, ip=10.0.0.62)   return transform_pyarrow.concat(tables)


(autoscaler +1m27s) [autoscaler] [worker-node-type-0|n2-standard-32] [us-west1-b] [on-demand] Launched 1 instances.


(MapWorker(MapBatches(process_data)->MapBatches(UpserterConcurrent)) pid=44830, ip=10.0.0.17) /home/ray/anaconda3/lib/python3.10/site-packages/ray/data/_internal/arrow_block.py:148: FutureWarning: promote has been superseded by mode='default'. [repeated 25x across cluster]
(MapWorker(MapBatches(process_data)->MapBatches(UpserterConcurrent)) pid=44830, ip=10.0.0.17)   return transform_pyarrow.concat(tables) [repeated 25x across cluster]
(MapWorker(MapBatches(process_data)->MapBatches(UpserterConcurrent)) pid=46641, ip=10.0.0.57) /home/ray/anaconda3/lib/python3.10/site-packages/ray/data/_internal/arrow_block.py:148: FutureWarning: promote has been superseded by mode='default'. [repeated 4x across cluster]
(MapWorker(MapBatches(process_data)->MapBatches(UpserterConcurrent)) pid=46641, ip=10.0.0.57)   return transform_pyarrow.concat(tables) [repeated 4x across cluster]
(MapWorker(MapBatches(process_data)->MapBatches(UpserterConcurrent)) pid=1215867) /home/ray/anaconda3/lib/python3.10/site-

...

- Aggregate 1:   0%|          | 0/2024 [00:00<?, ?it/s]

Shuffle Map 2:   0%|          | 0/2024 [00:00<?, ?it/s]

Shuffle Reduce 3:   0%|          | 0/2024 [00:00<?, ?it/s]

Running 0:   0%|          | 0/2024 [00:00<?, ?it/s]

(autoscaler +44m30s) [autoscaler] Downscaling node g-446513b32ebc40002 (node IP: 10.0.0.24) due to node idle termination.
(autoscaler +44m31s) [autoscaler] Cluster resized to {192 CPU, 0 GPU}.
{'sum(upsreted)': ' 35,167,920', 'sum(errors)': ' 0'}
0:43:10.921066
CPU times: user 1min 17s, sys: 9.55 s, total: 1min 27s
Wall time: 43min 10s


In [24]:
print(f"Upserted {summary['sum(upsreted)']:,} vectors, duration: {duration}. Failed to upsert {summary['sum(errors)']:,} vectors, see log for more details")

Upserted 35,167,920 vectors, duration: 0:43:10.921066. Failed to upsert 0 vectors, see log for more details


(autoscaler +45m0s) [autoscaler] Downscaling node g-fa67927eec76e0001 (node IP: 10.0.0.49) due to node idle termination.
(autoscaler +45m1s) [autoscaler] Cluster resized to {160 CPU, 0 GPU}.
